In [83]:
import sys
!{sys.executable} -m pip install plotly

     |████████████████████████████████| 26.5 MB 2.7 MB/s eta 0:00:01


In [27]:
import pandas as pd
import pandas
# monkeypatch using standard python json module

import json

pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

# monkeypatch using faster simplejson module
import simplejson
pd.io.json._json.loads = lambda s, *a, **kw: simplejson.loads(s)

# normalising (unnesting) at the same time (for nested jsons)
pd.io.json._json.loads = lambda s, *a, **kw: pandas.json_normalize(simplejson.loads(s))

# init functions

In [92]:
import plotly.graph_objects as go

def syscall_timeseires(df, processId, syscall_list=[], process_name=""):
    # processId needs to be in list
    if process_name:
        df = df[df['processName'] == process_name]
    df = df[df['processId'].isin(processId)]
    #df = df.explode('args')
    #df[['name', 'type', 'value']] = df['args'].apply(pd.Series)
    #df['args'] = df['eventName']+":"+df['args'].astype(str)
    #df = df[df['type']== 'const char*']
    if not syscall_list:
        syscall_list = list(df['eventName'].unique())
    data = []
    #process_name =  df.groupby("processName")["processId"].count().sort_values(ascending=False).idxmax()
    for sys in syscall_list:
        df2 = df[df['eventName'] == sys]
        data.append(go.Scatter(name=sys,x=df2['timestamp'],
                                        y=df2['eventName'],
                                        mode='markers',
                                        text=df2['args_print']))
    fig = go.Figure(data=data)
    fig.update_layout(title='Event Timeseries of {}'.format(process_name))
    fig.show()
    fig.write_html('graph.html')


# Working with the dataframe

In [30]:
df = pd.read_json('output.json', lines=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34995 entries, 0 to 34994
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            34995 non-null  datetime64[ns]
 1   processId            34995 non-null  int64         
 2   threadId             34995 non-null  int64         
 3   parentProcessId      34995 non-null  int64         
 4   hostProcessId        34995 non-null  int64         
 5   hostThreadId         34995 non-null  int64         
 6   hostParentProcessId  34995 non-null  int64         
 7   userId               34995 non-null  int64         
 8   mountNamespace       34995 non-null  int64         
 9   pidNamespace         34995 non-null  int64         
 10  processName          34995 non-null  object        
 11  hostName             34995 non-null  object        
 12  containerId          34995 non-null  object        
 13  eventId              34995 non-

In [37]:
df.groupby('hostName')['hostName'].count()

hostName
79a55092ebea     2305
ubuntu          32690
Name: hostName, dtype: int64

In [101]:
only_con.groupby('eventName')['eventName'].count()

eventName
access                  15
cap_capable            263
clone                    1
close                  173
execve                   3
fchown                   3
fstat                  224
getdents                 2
getdents64               2
lstat                    8
memfd_create             1
mount                   24
open                   811
openat                   8
prctl                   99
sched_process_exec       3
sched_process_exit       8
security_bprm_check      3
security_file_open     343
security_sb_mount       22
setgid                   1
setuid                   1
stat                   238
unlink                  49
Name: eventName, dtype: int64

In [42]:
pd.set_option('display.max_colwidth', None)

In [70]:
a = change_args([{'name': 'dirfd', 'type': 'int', 'value': 7}, {'name': 'argv', 'type': 'const char*const*', 'value': ['runc', 'init']}, {'name': 'flags', 'type': 'int', 'value': 4096}])

In [94]:
def change_args(x):
    s = ""
    for i in x:
        s+=json.dumps(i)
        s+='<br>'
    return s

In [95]:
df['args_print'] = df['args'].apply(change_args)

In [96]:
df[df['eventName'].isin(['execveat','memfd_create'])].T

,13596,26019
timestamp,2022-01-11 11:27:19.694318349,2022-01-11 11:27:20.035990039
processId,16625,8
threadId,16625,8
parentProcessId,16624,1
hostProcessId,16625,16662
hostThreadId,16625,16662
hostParentProcessId,16616,16628
userId,0,0
mountNamespace,4026531840,4026532950
pidNamespace,4026531836,4026532953


In [97]:
only_con = df[df['hostName'] == '79a55092ebea']

In [102]:
syscall_timeseires(only_con, [8])

In [100]:
syscall_timeseires(only_con, [1])